# Create Users

## Présentation
L'objectif de ce script est d'agréger les données relatives aux utilisateurs qui ont envoyé un tweet et de les stocker en base de données afin qu'elles puissent être utilisées ultérieurement pour d'autres analyses, telles que la classification ou le clustering d'utilisateurs en fonction de s'ils sont atypiques ou non.

Que fait-il ?
1. Récupère le dernier tweet qui a été envoyé. Les tweets étant figés, cette récupération permet d'avoir des durées cohérentes, pas trop importantes.
2. On agrège les données des utilisateurs en fonctions des tweets qu'ils ont envoyés.
3. Calcul d'attributs supplémentaire qui pourront être utilisé pour classifier et créer des clusters d'utilisateur


Pour réaliser la classification et le clustering, nous avons décidé de définir un utilisateur de twitter par :
- S'il s'agit ou non d'un compte vérifié
- Le nombre total de tweet envoyé
- ratio friend/followers
- Le nombre total de tweet aimés par l'utilisateur
- l'âge du compte
- Le nombre moyen de RT qu'il possède sur ces tweets
- Fréquence d'envois de message par jour
- Ratio du nombre de hastag par tweet
- Degré d'agressivité
- ça visibilité
- le ratio du nombre de messages sensible qu'il a envoyé
- ...

## Le code
On importe les différentes bibliothèques nécessaires.

In [ ]:
import pymongo

Connexion à la base de données MongoDB et plus précisément, aux deux collections utiles pour la classification et clustering:
- ``dataset`` dédiée à stocker les différents tweets initiaux
- ``users`` dédiée à stocker les différents utilisateurs de Twitter

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
tweet_collection = db["dataset"]
user_collection = db["users"]

On supprime toute la collection pour supprimer par la même occasion les données qu'elle contient.

In [ ]:
user_collection.drop()

Récupération du dernier tweet de la base de données pour pouvoir par la suite estimer l'âge d'un compte utilisateur à partir de cette date et non celle d'aujourd'hui pour éviter d'avoir des valeurs énormes, car les tweets sont figés.

In [ ]:
pipeline = [
    {
        "$sort" : {
            "created_date" : -1
        }
    },
    {
        "$limit": 1
    }
]
last_tweet_published = tweet_collection.aggregate(pipeline)
last_tweet_published = list(last_tweet_published)[0]
last_tweet_published

Isoler et regrouper les données des utilisateurs de Twitter pour ensuite calculer des attributs supplémentaires et les stocker en base de données.

In [ ]:
users = tweet_collection.aggregate([
    {
        "$group": {
            "_id": "$user.id",
            "tweets": { "$push": "$$ROOT" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "user_id": "$_id",
            "tweets": 1
        }
    },
    {
        "$sort" : {
            "tweets.created_date" : 1
        }
    },
    {
        "$addFields": {
            "verified": {"$last": "$tweets.user.verified"},
            "statuses_count": {"$last": "$tweets.user.statuses_count"},
            "favourites_count": {"$last": "$tweets.user.favourites_count"},
            "friends_count":{ "$last": "$tweets.user.friends_count" },
            "followers_count":{ "$last": "$tweets.user.followers_count" },
            "age_account": {
                "$divide": [
                  {
                    "$subtract": [last_tweet_published['created_date'], {"$last": "$tweets.user.created_at"}]
                  },
                  86400000 # nombre de millisecondes dans une journée
                ]
            },
            "last_tweet_published_id": {"$last": "$tweets._id"},
             "ratio_friends_followers": {
                "$cond": [
                { "$eq": [ { "$last": "$tweets.user.followers_count" }, 0 ] },
                -1,
                {
                    "$divide": [
                    { "$last": "$tweets.user.friends_count" },
                    { "$last": "$tweets.user.followers_count" }
                    ]
                }
                ]
            }
        }
    },
    {
        "$addFields": {
            "avg_tweet_length": {
                "$avg": {
                    "$map": {
                        "input": "$tweets",
                        "as": "tweet",
                        "in": { "$strLenCP": "$$tweet.text" }
                    }
                }
            },
             "tweet_frequency":{
                    "$divide": [
                           "$statuses_count", "$age_account" 
                        ]
                    },
            "avg_retweet_count": {
                "$avg": "$tweets.retweet_count"
            },
            "hashtag_frequency":{
                "$avg":{
                    "$size":"$tweets.entities.hashtags"
                }
            },
              "frequency_friend_hour":{
                "$divide":[
                  {
                    "$sum":{
                        "$add":[
                            { "$size": { "$ifNull": ["$tweets.entities.media", []] }},
                            { "$size":"$tweets.entities.urls"}
                        ]
                    }
                  },
                  {
                      "$multiply": [ "$age_account", 24 ] 
                  }
                ]
            },
            "nb_sensitive_tweets": {
                "$sum": {
                    "$map": {
                        "input": "$tweets",
                        "as": "tweet",
                        "in": { 
                            "$cond": [
                                {'$eq': [{"$toString": "$$tweet.possibly_sensitive"}, "true"]},
                                1,
                                0
                            ]
                        }
                    } 
                }
            }
        }
    },
    {
         "$addFields":{
              "Ap":{  #Dégré d'aggressivité
                "$divide":[
                  {
                      "$add": [
                          { "$divide":["$tweet_frequency",24] },
                          "$frequency_friend_hour"
                        ]
                  },
                  350
                ]
            },
            "visibility": {
                "$divide": [
                    {
                        "$add": [
                        {
                            "$multiply": [
                                {
                                    "$avg": {
                                        "$map": {
                                            "input": "$tweets",
                                            "as": "tweet",
                                            "in": {
                                            "$avg": {
                                                "$map": {
                                                "input": "$$tweet.entities.user_mentions",
                                                "as": "mention",
                                                "in": { "$strLenCP": "$$mention.screen_name" }
                                                }
                                            }
                                            }
                                        }
                                    }
                                },
                                {
                                    "$avg": {"$size" : "$tweets.entities.user_mentions"}
                                }
                            ]
                        },
                        {
                            "$multiply": [
                                {
                                    "$avg": {
                                        "$map": {
                                            "input": "$tweets",
                                            "as": "tweet",
                                            "in": {
                                            "$avg": {
                                                "$map": {
                                                "input": "$$tweet.entities.hashtags",
                                                "as": "hashtags",
                                                "in": { "$strLenCP": "$$hashtags.text" }
                                                }
                                            }
                                            }
                                        }
                                    }
                                },
                                {
                                    "$avg": {"$size":"$tweets.entities.hashtags"}
                                }
                            ]
                        }
                        ]
                    }, 140
                ]
            },
            "ratio_sensitive_tweets": {
                "$cond": [
                    { "$ne": [{ "$size": "$tweets" }, 0] },
                    { 
                        "$divide": [
                            "$nb_sensitive_tweets", 
                            { "$size": "$tweets" }
                        ] 
                    },
                    -1
                ]
            },
         }
    },
    {
        "$project": {
            "user_id": 1,
            "tweet_ids": "$tweets._id",
            "avg_tweet_length": 1,
            "tweet_frequency": 1,
            "last_tweet_published_id": 1,
            "verified": 1,
            "statuses_count": 1,
            "favourites_count": 1,
            "friends_count":1,
            "followers_count":1,
            "ratio_friends_followers": 1,
            "age_account": 1,
            "avg_retweet_count": 1,
            "hashtag_frequency":1,
            "visibility": 1,
            "Ap":1,
            "ratio_sensitive_tweets": 1,
            "nb_sensitive_tweets": 1,
            "_id": 0
        }
    }
])

user_collection.insert_many(list(users))



In [ ]:
pipeline = [
    {"$unwind": "$tweet_ids"},
    {"$group": {"_id": "$user_id", "tweet_ids_count": {"$sum": 1}}},
    {"$sort":{"tweet_ids_count":-1}}
]

# Execute the aggregation pipeline
result = user_collection.aggregate(pipeline)

# Print the results
for doc in result:
    print("user_id: {}, tweet_ids_count: {}".format(doc["_id"], doc["tweet_ids_count"]))